## REad the data

In [81]:
import pandas as pd
from sodapy import Socrata
import numpy as np

In [82]:

import pandas as pd
from sodapy import Socrata

client = Socrata("data.texas.gov", None)

results = client.get("ubdr-4uff", limit=300000)

# Convert to pandas DataFrame
df1 = pd.DataFrame.from_records(results)

In [83]:
df = df1

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259452 entries, 0 to 259451
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   complaint_number          259452 non-null  object
 1   respondent_name           259452 non-null  object
 2   complainant_role          259452 non-null  object
 3   reason                    259446 non-null  object
 4   complaint_confirmed_code  259447 non-null  object
 5   disposition               258414 non-null  object
 6   received_date             259452 non-null  object
 7   closed_date               259452 non-null  object
 8   complaint_type            259451 non-null  object
 9   coverage_type             259452 non-null  object
 10  coverage_level            259452 non-null  object
 11  respondent_id             259452 non-null  object
 12  respondent_role           259450 non-null  object
 13  respondent_type           259452 non-null  object
 14  comp

In [85]:
# Drop missing rows in disposition coluMN
df = df.dropna(subset=['disposition'])

In [ ]:

# get counts of disposition values in descending order
disposition_counts = df['disposition'].value_counts().sort_values(ascending=False)
print(disposition_counts)


disposition
Contract Language/Legal Issue                                        25698
No Jurisdiction                                                      23533
Information Furnished                                                17925
Question of Fact                                                     12215
Information Furnished; Contract Language/Legal Issue                  8085
                                                                     ...  
Referred To; Question of Fact; Company Position Upheld                   1
Agent Resp Recvd; Policy Issued/Restored; Corrective Action Taken        1
Referred To; Record Only; Insufficient Information                       1
Information Furnished; Referred To; Claim Settled                        1
Premium Refunded; Claim Settled; Corrective Action Taken                 1
Name: count, Length: 4068, dtype: int64


### Disposition column

In [ ]:


# Define keywords for categorizing dispositions as 'verdict won' or 'verdict not won'
verdict_won_keywords = ["Claim Settled", "Additional Monies Received"]
verdict_not_won_keywords = ["No Jurisdiction", "Company Position Upheld", "Insufficient Information", "Contract Language/Legal Issue", "Question of Fact"]

# Function to categorize each disposition based on the presence of keywords
def categorize_verdict(disposition):
    if any(keyword in disposition for keyword in verdict_won_keywords):
        return "Yes"
    elif any(keyword in disposition for keyword in verdict_not_won_keywords):
        return "No"
    else:
        return "Uncategorized"  # For entries that don't match any keywords

# Apply categorization to the disposition data
df['verdict_passed'] = df['disposition'].apply(categorize_verdict)

# Drop rows where the disposition was categorized as "Uncategorized"
df = df[df['verdict_passed'] != "Uncategorized"]

# replace disposition column with verdict_passed column and drop verdict_passed
df['disposition'] = df['verdict_passed']
df = df.drop(columns=['verdict_passed'])


C:\Users\nbngc\AppData\Local\Temp\ipykernel_68392\3181898158.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['verdict_passed'] = df['disposition'].apply(categorize_verdict)


In [88]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 206807 entries, 4 to 259450
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   complaint_number          206807 non-null  object
 1   respondent_name           206807 non-null  object
 2   complainant_role          206807 non-null  object
 3   reason                    206806 non-null  object
 4   complaint_confirmed_code  206802 non-null  object
 5   disposition               206807 non-null  object
 6   received_date             206807 non-null  object
 7   closed_date               206807 non-null  object
 8   complaint_type            206806 non-null  object
 9   coverage_type             206807 non-null  object
 10  coverage_level            206807 non-null  object
 11  respondent_id             206807 non-null  object
 12  respondent_role           206807 non-null  object
 13  respondent_type           206807 non-null  object
 14  complaina

# Drop unnecessary columns

In [89]:
# drop the columns
# count of individual in column respondant type
df['respondent_type'].value_counts()

# drop columns keyword, respondent_id,complaint_number
df = df.drop(columns=['keyword','respondent_id','complaint_number'])
 

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 206807 entries, 4 to 259450
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   respondent_name           206807 non-null  object
 1   complainant_role          206807 non-null  object
 2   reason                    206806 non-null  object
 3   complaint_confirmed_code  206802 non-null  object
 4   disposition               206807 non-null  object
 5   received_date             206807 non-null  object
 6   closed_date               206807 non-null  object
 7   complaint_type            206806 non-null  object
 8   coverage_type             206807 non-null  object
 9   coverage_level            206807 non-null  object
 10  respondent_role           206807 non-null  object
 11  respondent_type           206807 non-null  object
 12  complainant_type          206807 non-null  object
 13  involved_party_type       186364 non-null  object
dtypes: object

In [91]:
# Trim whitespace from all columns with string data
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [92]:
# Convert all columns to propercase if they are strings
df = df.apply(lambda x: x.str.title() if x.dtype == "object" else x)

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 206807 entries, 4 to 259450
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   respondent_name           206807 non-null  object
 1   complainant_role          206807 non-null  object
 2   reason                    206806 non-null  object
 3   complaint_confirmed_code  206802 non-null  object
 4   disposition               206807 non-null  object
 5   received_date             206807 non-null  object
 6   closed_date               206807 non-null  object
 7   complaint_type            206806 non-null  object
 8   coverage_type             206807 non-null  object
 9   coverage_level            206807 non-null  object
 10  respondent_role           206807 non-null  object
 11  respondent_type           206807 non-null  object
 12  complainant_type          206807 non-null  object
 13  involved_party_type       186364 non-null  object
dtypes: object

In [94]:
label_enco_cols = ["respondent_name", "reason", "involved_party_type"]
one_enco_cols = ["complainant_role", "complaint_confirmed_code","complaint_type","coverage_type","coverage_level","respondent_role","respondent_type", "complainant_type"]

# Remove the text ", The" from respondent_name column if ends in ", The"
df['respondent_name'] = df['respondent_name'].str.rstrip(', The')

# for label_enco_cols do label encoding for one_enco_cols do one hot encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# Label Encoding
for col in label_enco_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# One Hot Encoding
df = pd.get_dummies(df, columns=one_enco_cols, drop_first=True)


In [95]:
# implement MCA on the data
# from prince import MCA
# mca = MCA(n_components=2)
# mca.fit(df)
# mca.plot_coordinates(df, figsize=(15, 15))


In [97]:
df.head()

,respondent_name,reason,disposition,received_date,closed_date,involved_party_type,complainant_role_Attorney,complainant_role_Beneficiary,complainant_role_Consumer (Non-Insured),complainant_role_Contracted Provider,...,respondent_role_Texas Fair Insurance Plan,respondent_role_Third Party Administrators,respondent_role_Twia,respondent_role_U.S. Domiciled Insurer,respondent_role_Unauthorized Activity,respondent_role_Utilization Review Agent,respondent_role_Viatical Settlement Com/Broker,respondent_role_Wc Healthcare Provider Ntwk,respondent_type_Organization,complainant_type_Org
4,1754,6514,No,2012-06-13T00:00:00.000,2012-07-17T00:00:00.000,6117,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
5,7002,2469,No,2012-06-18T00:00:00.000,2012-07-30T00:00:00.000,6117,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
6,8949,6514,Yes,2012-02-27T00:00:00.000,2012-07-02T00:00:00.000,5453,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7,1197,5121,No,2012-06-19T00:00:00.000,2012-08-06T00:00:00.000,6117,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
8,6816,6514,No,2011-10-03T00:00:00.000,2012-07-20T00:00:00.000,6117,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [98]:
df.describe()

,respondent_name,reason,involved_party_type
count,206807.000000,206807.000000,206807.000000
mean,4476.191705,4468.123618,4341.552143
std,3165.615335,1558.953874,1634.898422
min,0.000000,0.000000,0.000000
25%,1197.000000,3539.000000,3168.000000
50%,4184.000000,4576.000000,4843.000000
75%,7909.000000,5538.000000,5963.000000
max,9589.000000,6537.000000,6117.000000


In [ ]:
# write disposition counts to a csv file
df.to_csv('df.csv')

## Train Test Split